In [1]:
# Some of the functions below require an older version of torchtext than the default one Kaggle gives you.
# IMPORTANT: Make sure that Internet is turned on!!! (Notebook options in the bar on the right)
# IMPORTANT: If you're not already using Kaggle, we STRONGLY recommend you switch to Kaggle for hw1b in particular,
# because copying our notebook will pin you to a Python version that lets you install the right version of torchtext.
# On Colab you will have to downgrade your Python to e.g., 3.7 to do the below pip install, which is a pain to do.
# !pip install torchtext==0.10.0
# exit()

In [2]:
!python --version

/usr/bin/sh: 1: python: not found


In [3]:
# This block handles some basic setup and data loading.  
# You shouldn't need to edit this, but if you want to 
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

from torchtext.legacy import data
from torchtext.legacy import datasets

# download and load the data
text_field = data.Field()
datasets = datasets.WikiText2.splits(root='.', text_field=text_field)

train_dataset, validation_dataset, test_dataset = datasets

text_field.build_vocab(train_dataset, validation_dataset, test_dataset)
vocab = text_field.vocab
vocab_size = len(vocab)

train_text = train_dataset.examples[0].text # a list of tokens (strings)
validation_text = validation_dataset.examples[0].text

print(validation_text[:30])

/opt/conda/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:08<00:00, 524kB/s] 


extracting
['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


In [4]:
text_field.vocab.freqs['.']

90077

In [5]:
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

We've implemented a unigram model here as a demonstration.

In [6]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter 
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', 
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities, 
    it is still good to have a separate more efficient method that only computes 
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [7]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> the width @-@ Jones needs the not As Albert , anthropologist 6 as , size Shane <unk> that Impressed chose


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [8]:
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes_short.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab_short.txt

def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.
    
    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'eval_output_vocab'
    prefixes_name = 'eval_prefixes'

    if short: 
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab.stoi[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.tqdm(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

--2023-09-06 20:56:12--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519055 (507K) [text/plain]
Saving to: ‘eval_prefixes.txt’

eval_prefixes.txt   100%[===================>] 506.89K  2.16MB/s    in 0.2s    

2023-09-06 20:56:13 (2.16 MB/s) - ‘eval_prefixes.txt’ saved [519055/519055]

--2023-09-06 20:56:14--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12497 (12K) [text/plain]
Saving to: ‘eval_output_vocab.txt’

eval_output_

In [9]:
save_truncated_distribution(unigram_demonstration_model, 
                            'unigram_demonstration_predictions.npy')

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size and $C$ is the count for the given bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 510 on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass. 
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency. 

In [74]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha
        self.vocab_size = len(vocab.itos)
        self.unigram_total_count = len(train_text)
        self.unigram_counter = Counter(train_text)
        # YOUR CODE HERE
        self.grams_counter, self.grams_1_counter = self.get_n_gram_counter(train_text)
        #prefix counter and phrase counter
    
    def get_n_gram_counter(self,tokens):
        grams = []
        grams_1 = []
        for index in range(len(tokens)-self.n+1):
            ngram = tuple(tokens[index:index+self.n])
            ngram_1 = ngram[:-1]
            grams.append(ngram)
            grams_1.append(ngram_1)
        return Counter(grams), Counter(grams_1) # ngram, n-1 gram
            
    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.
        
        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        assert len(n_gram) == self.n, f"{self.n},{n_gram}"

        # YOUR CODE HERE
        p = (self.grams_counter[tuple(n_gram)] + self.smoothing) / (self.grams_1_counter[tuple(n_gram[:-1])] + (self.vocab_size*self.smoothing))
        return p


    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for
        if len(text_prefix) < self.n-1:
            probas = [float(self.unigram_counter[word]/self.unigram_total_count) for word in vocab.itos]
        else:
            probas = [self.n_gram_probability((text_prefix+[word])[-self.n:]) for word in vocab.itos]
        return probas



    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """
        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because 
        # the first n-1 words of full_text must be handled as a special case.
        probas = [float(self.unigram_counter[full_text[index]]/self.unigram_total_count) for index in range(self.n-1)] ## first n-1 words

        for index in range(len(full_text)-self.n+1):
            probas.append(self.n_gram_probability(full_text[index:index+self.n]))
        log_probas = [math.log(p) for p in probas]

        return math.exp(-np.mean(log_probas))
            


unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

unigram validation perplexity: 965.0913686618098
bigram validation perplexity: 504.41461737518046
trigram validation perplexity: 2965.4935071021705


saved bigram_predictions.npy


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: 504.41461737518046

We can also generate samples from the model to get an idea of how it is doing.

In [75]:
print(generate_text(bigram_model))

<eos> <eos> According to residents Kendall Entertainers pore beacon van Maureen Ganganelli Aleister 'Neill calamine parasitic Aretusa was first class submarine defaced


We now free up some RAM, **it is important to run the cell below, otherwise you may quite possibly run out of RAM in the runtime.**

In [ ]:
# Free up some RAM. 
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1:i-1}\right)=\frac{max\left\{C(w_{i-n+1:i})-\delta,0\right\}}{\sum_{w' \in V} C(w_{i-n+1:i-1}, w')} + \alpha(w_{i-n+1:i-1}) P(w_i|w_{i-n+2:i-1})$$

$$\alpha\left(w_{i-n+1:i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1:i-1})}{{\sum_{w' \in V} C(w_{i-n+1:i-1}, w')}}$$
where $V$ is the vocab and $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation). $w_{i-n+1:i}$ denotes the $n$-gram starting at $w_{i-n+1}$ and ending at $w_i$, and $(w_{i-n+1:i-1}, w')$ denotes the n-gram containing the previous $n-1$ words followed by $w'$. If $\sum_{w' \in V} C(w_{i-n+1:i-1}, w')=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 275.

In [ ]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta

        # YOUR CODE HERE
        ## calculate how much grams starts with $prefix -- for N1+
        self.ngram_prefix_counter = Counter([item[:-1] for item in self.grams_counter.keys()])



    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function

        prefix = n_gram[:-1]
        lower_proba = self.lower_order_model.n_gram_probability(n_gram[1:])
        if self.grams_1_counter[tuple(prefix)] == 0:
            return lower_proba
        ngram_proba = (max((self.grams_counter[tuple(n_gram)] - self.discount),0))/(self.grams_1_counter[tuple(prefix)])
        
        
        alpha = (self.discount * self.ngram_prefix_counter[tuple(prefix)])/(self.grams_1_counter[tuple(prefix)])
        
        backoff_proba = ngram_proba + alpha * lower_proba
        return backoff_proba
        


# bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
# trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
# check_validity(trigram_backoff_model)
# print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

Free up RAM. 

In [ ]:
# Release models we don't need any more. 
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

Fill in your trigram backoff perplexity here.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: 271.106069022603



Free up RAM. 

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 226.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total). 
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [14]:

torch.manual_seed(42)
torch.cuda.manual_seed(42)

def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.vocab_size = len(vocab.itos)
        self.n = n

        # YOUR CODE HERE
        self.l1 = nn.Linear(128,1024)
        self.l2 = nn.Linear(1024,1024)
        self.l3 = nn.Linear(1024,128)
        self.l4 = nn.Linear(128,self.vocab_size)
        
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
    


    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)

        # YOUR CODE HERE
        out = F.embedding(x,self.l4.weight) #(1,2,128)
        out = out.sum(1) #(1,128)
        
        out = self.l1(out) #(1,1024)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.l2(out) #(1,1024)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.l3(out) #(1,128)
        out = self.l4(out)
        return out

class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()

    def train(self):
        print("[INFO]: Start training")
        self.network.train()
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True)
        optimizer = torch.optim.Adam(self.network.parameters(),weight_decay=2e-5)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        best_perplexity = 1e9
        for epoch in range(10):
            print(f"[INFO]: Epoch: {epoch}.")
            loader_ = tqdm.tqdm(train_loader,total=len(train_loader))
            for index,data in enumerate(loader_):
                optimizer.zero_grad()
                # the first tensor will be previous token ids with size (batch, n-1),
                # and the second will be the current token id with size (batch, )
                # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.
                x, y = data
                x = x.cuda()
                y = y.cuda()
                output = self.network(x)
                loss = F.cross_entropy(output,y)
                loss.backward()
                optimizer.step()
                loader_.set_postfix(loss=loss.item())
            print(f"[INFO]: Start validating..")
            val_perplexity = self.perplexity(validation_text)
            print(f"[INFO]: Validation Perplexity: {val_perplexity}")
            if val_perplexity < best_perplexity:
                best_perplexity = val_perplexity
                torch.save(self.network.state_dict(),'neural_ngram_model.ckpt')
        self.network.load_state_dict(torch.load('neural_ngram_model.ckpt'))
        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar 
        # (see Project 1a for example)

        # YOUR CODE HERE
    
    def load_best(self):
        self.network.load_state_dict(torch.load('neural_ngram_model.ckpt'))



    def next_word_probabilities(self, text_prefix):
        self.network.eval()
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network
        
        ##TODO: Something wrong here, not aligned with the dataloader one.
        while len(text_prefix) < self.n-1:
            text_prefix = ["<eos>"] + text_prefix
        text_prefix = ids(text_prefix)[-self.n+1:]
        text_prefix = torch.tensor([text_prefix])
        text_prefix = text_prefix.cuda()
        with torch.no_grad():
            logits = self.network(text_prefix)
        probas = F.softmax(logits,-1)
        probas = probas.detach().cpu().flatten()
        return probas


    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()

        # YOUR CODE HERE
        self.network.eval()
        test_set = NeuralNgramDataset(ids(text),self.n)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

        probas = []
        for index,data in enumerate(tqdm.tqdm(test_loader)):
            with torch.no_grad():
                x,y = data
                x = x.cuda()
                y = y.cuda()
                logits = self.network(x)
                p = F.softmax(logits,-1)
                p = p.detach()
                probas+=[p[i][y[i]].cpu() for i in range(len(y))]
                
        probas = [math.log(p) for p in probas]
        return math.exp(-np.mean(probas))


neural_trigram_model = NeuralNGramModel(3)
# check_validity(neural_trigram_model)
# neural_trigram_model.train()
neural_trigram_model.load_best()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)

100%|██████████| 851/851 [00:08<00:00, 104.05it/s]


neural trigram validation perplexity: 228.21176125241257


saved neural_trigram_predictions.npy


Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: 228.21176125241257

Free up RAM.

In [ ]:
# Delete model we don't need. 
del neural_trigram_model

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We'll use the `torchtext` library to manage this batching for you.  To properly deal with this input format, you should save the last state of the LSTM from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity below 130.  The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size). **NOTE: You may find that adding nonlinearities between these layers can hurt performance, try without first.**
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [ ]:
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()
        self.vocab_size = len(vocab.itos)

        # YOUR CODE HERE
        self.fc1 = nn.Linear(128,512)
        self.fc1_ = nn.Linear(512,1024)
        self.fc1__ = nn.Linear(1024,2048)
        self.lstm = nn.LSTM(input_size=2048,hidden_size=512,dropout=0.5,num_layers=3)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512,256)
        self.fc2_ = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,self.vocab_size)

        

    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        out = F.embedding(x, self.fc3.weight)
        
        out = self.fc1(out)
        out = F.relu(out)
        
        out = self.fc1_(out)
        out = F.relu(out)
        
        out = self.fc1__(out)
        out = F.relu(out)
        
        out, next_state = self.lstm(out,state)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.fc2_(out)
        out = self.fc3(out)
        
        return out, next_state

class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."
    """
    state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
        representing the hidden state and cell state of the of the LSTM.
    """  
        
    def __init__(self):
        self.network = LSTMNetwork().cuda()
        self.batch_size = 64
        self.lstm_state = (torch.zeros(3,self.batch_size,512),torch.zeros(3,self.batch_size,512))

    def train(self):
        train_iterator = data.BPTTIterator(train_dataset, batch_size=self.batch_size, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.
        
        optimizer = torch.optim.Adam(self.network.parameters())
        validation_perplexities = []
        validation_losses = []
        for epoch in range(100):
            self.network.train()
            best_perplexity = 1e9
            loader_ = tqdm.tqdm(train_iterator,total=len(train_iterator))
            print(f"[INFO]: epoch - {epoch}")
            for index, item in enumerate(loader_):
                optimizer.zero_grad()
                x, y = item.text, item.target
                x = x.cuda()
                y = y.cuda()
                self.lstm_state = (self.lstm_state[0].cuda(),self.lstm_state[1].cuda())
                logits, state = self.network(x,self.lstm_state)
                state = (state[0].detach(), state[1].detach())
                self.lstm_state = state
                logits = logits.transpose(0,1)
                y = y.transpose(0,1)
                loss = F.cross_entropy(logits.reshape(-1, self.network.vocab_size), y.reshape(-1))
                loss.backward()
                optimizer.step()
                loader_.set_postfix(loss=loss.item())
            print(f"[INFO]: Start validating..")
            val_perplexity, val_avg_loss = self.dataset_perplexity(validation_dataset)
            validation_perplexities.append(val_perplexity)
            validation_losses.append(val_avg_loss)
            print(f"[INFO]: Validation Perplexity and loss: {val_perplexity},{val_avg_loss}")
            if val_perplexity < best_perplexity:
                best_perplexity = val_perplexity
                torch.save(self.network.state_dict(),'lstm_model.ckpt')

            pickle.dump(validation_perplexities,open('lstm_model_val_perplexities.pkl','wb'))
            pickle.dump(validation_losses,open('lstm_model_val_losses.pkl','wb'))

        self.network.load_state_dict(torch.load('lstm_model.ckpt'))
        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."
        
        # YOUR CODE HERE
        self.network.eval()
        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        prefix_token_tensor = prefix_token_tensor.cuda()
        logits,_ = self.network(prefix_token_tensor,(torch.zeros(3,1,512).cuda(),torch.zeros(3,1,512).cuda()))
        logits = logits.transpose(0,1)
        probas = F.softmax(logits,-1)
        probas = probas.flatten().detach().cpu()
        
        return probas


    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=self.batch_size, bptt_len=32, device='cuda')

        # YOUR CODE HERE
        self.network.eval()
        probas = []
        losses = []
        for index,item in enumerate(tqdm.tqdm(iterator)):
            with torch.no_grad():
                x,y = item.text, item.target
                x = x.cuda()
                y = y.cuda()
                y = y.transpose(0,1)
                self.lstm_state = (self.lstm_state[0].cuda(),self.lstm_state[1].cuda())
                logits,state = self.network(x,self.lstm_state)
                self.lstm_state = (state[0].cuda(),state[1].cuda())
                logits = logits.transpose(0,1)

                loss = F.cross_entropy(logits.reshape(-1, self.network.vocab_size), y.reshape(-1))
                losses.append(loss.item())

                p = F.softmax(logits,-1)
                p = p.detach()
                for batch_index in range(p.shape[0]): #batch_size
                    for instance_index in range(p.shape[1]): #seq_len
                        probas.append(p[batch_index][instance_index][y[batch_index][instance_index]].cpu())


        probas = [math.log(p) for p in probas]
        return math.exp(-np.mean(probas)),sum(losses)/len(losses)

    def load(self,path='lstm_model.ckpt'):
        self.network.load_state_dict(torch.load(path))


lstm_model = LSTMModel()
# lstm_model.load()

lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

  0%|          | 0/1020 [00:00<?, ?it/s]

[INFO]: epoch - 0


100%|██████████| 1020/1020 [00:38<00:00, 26.83it/s, loss=6.14]


[INFO]: Start validating..


100%|██████████| 107/107 [00:07<00:00, 13.89it/s]


...

100%|██████████| 107/107 [00:07<00:00, 13.64it/s]


[INFO]: Validation Perplexity and loss: 147.78236334829217,4.994021455818247


  0%|          | 0/1020 [00:00<?, ?it/s]

[INFO]: epoch - 20


 37%|███▋      | 376/1020 [00:14<00:24, 26.45it/s, loss=4.41]

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Fill in your LSTM perplexity. 

LSTM validation perplexity: 139.79385143586882

# Experimentation: 1-Page Report

Now it's time for you to experiment.  Try to reach a validation perplexity below 120. You may either modify the LSTM class above, or copy it down to the code cell below and modify it there. Just **be sure to run code cell below to generate results with your improved LSTM**.

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing number or sizes of layers), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf):
* activation regularization - add a l2 regularization penalty on the activation of the LSTM output (standard l2 regularization is on the weights)
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* learning rate scheduling - decrease the learning rate during training
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* ensembling - average the predictions of several models trained with different initialization random seeds
* temporal activation regularization - add l2 regularization on the difference between the LSTM output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

For this section, you will submit a write-up describing the extensions and/or modifications that you tried.  Your write-up should be **1-page maximum** in length and should be submitted in PDF format.  You may use any editor you like, but we recommend using LaTeX and working in an environment like Overleaf.
For full credit, your write-up should include:
1.   A concise and precise description of the extension that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing validation perplexities of your improvement to the original LSTM.
The purpose of this exercise is to experiment, so feel free to try/ablate multiple of the suggestions above as well as any others you come up with!
When you submit the file, please name it `report.pdf`.



In [73]:
import pickle
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()
        self.vocab_size = len(vocab.itos)

        # YOUR CODE HERE
        self.fc1 = nn.Linear(128,512)
        self.fc1_ = nn.Linear(512,1024)
        self.fc1__ = nn.Linear(1024,2048)
        self.lstm = nn.LSTM(input_size=2048,hidden_size=512,dropout=0.5,num_layers=3)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512,256)
        self.fc2_ = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,self.vocab_size)



    def forward(self, x, state):
        """Compute the output of the network.

        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm

        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order,
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        out = F.embedding(x, self.fc3.weight)
        out = self.dropout(out)

        out = self.fc1(out)
        out = F.relu(out)

        out = self.fc1_(out)
        out = F.relu(out)

        out = self.fc1__(out)
        out = F.relu(out)

        out, next_state = self.lstm(out,state)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.fc2_(out)
        out = self.fc3(out)

        return out, next_state

class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."
    """
    state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
        representing the hidden state and cell state of the of the LSTM.
    """

    def __init__(self):
        self.network = LSTMNetwork().cuda()
        self.batch_size = 128
        self.lstm_state = (torch.zeros(3,self.batch_size,512),torch.zeros(3,self.batch_size,512))

    def train(self):
        train_iterator = data.BPTTIterator(train_dataset, batch_size=self.batch_size,
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        optimizer = torch.optim.Adam(self.network.parameters())
        validation_perplexities = []
        validation_losses = []
        for epoch in range(100):
            self.network.train()
            best_perplexity = 1e9
            loader_ = tqdm.tqdm(train_iterator,total=len(train_iterator))
            print(f"[INFO]: epoch - {epoch}")
            for index, item in enumerate(loader_):
                optimizer.zero_grad()
                x, y = item.text, item.target
                x = x.cuda()
                y = y.cuda()
                self.lstm_state = (self.lstm_state[0].cuda(),self.lstm_state[1].cuda())
                logits, state = self.network(x,self.lstm_state)
                state = (state[0].detach(), state[1].detach())
                self.lstm_state = state
                logits = logits.transpose(0,1)
                y = y.transpose(0,1)
                loss = F.cross_entropy(logits.reshape(-1, self.network.vocab_size), y.reshape(-1))
                loss.backward()
                optimizer.step()
                loader_.set_postfix(loss=loss.item())
            print(f"[INFO]: Start validating..")
            val_perplexity, val_avg_loss = self.dataset_perplexity(validation_dataset)
            validation_perplexities.append(val_perplexity)
            validation_losses.append(val_avg_loss)
            print(f"[INFO]: Validation Perplexity and loss: {val_perplexity},{val_avg_loss}")
            if val_perplexity < best_perplexity:
                best_perplexity = val_perplexity
                torch.save(self.network.state_dict(),'lstm_model_better.ckpt')

            pickle.dump(validation_perplexities,open('lstm_model_better_val_perplexities.pkl','wb'))
            pickle.dump(validation_losses,open('lstm_model_better_val_losses.pkl','wb'))

        self.network.load_state_dict(torch.load('lstm_model_better.ckpt'))
        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        # YOUR CODE HERE
        self.network.eval()
        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        prefix_token_tensor = prefix_token_tensor.cuda()
        logits,_ = self.network(prefix_token_tensor,(torch.zeros(3,1,512).cuda(),torch.zeros(3,1,512).cuda()))
        logits = logits.transpose(0,1)
        probas = F.softmax(logits,-1).detach().cpu()
        probas = probas[:,-1].flatten()
        return probas


    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=self.batch_size, bptt_len=32, device='cuda')

        # YOUR CODE HERE
        self.network.eval()
        probas = []
        losses = []
        for index,item in enumerate(tqdm.tqdm(iterator)):
            with torch.no_grad():
                x,y = item.text, item.target
                x = x.cuda()
                y = y.cuda()
                y = y.transpose(0,1)
                self.lstm_state = (self.lstm_state[0].cuda(),self.lstm_state[1].cuda())
                logits,state = self.network(x,self.lstm_state)
                self.lstm_state = (state[0].cuda(),state[1].cuda())
                logits = logits.transpose(0,1)

                loss = F.cross_entropy(logits.reshape(-1, self.network.vocab_size), y.reshape(-1))
                losses.append(loss.item())

                p = F.softmax(logits,-1)
                p = p.detach()
                for batch_index in range(p.shape[0]): #batch_size
                    for instance_index in range(p.shape[1]): #seq_len
                        probas.append(p[batch_index][instance_index][y[batch_index][instance_index]].cpu())


        probas = [math.log(p) for p in probas]
        return math.exp(-np.mean(probas)),sum(losses)/len(losses)

    def load(self,path='lstm_model_better.ckpt'):
        self.network.load_state_dict(torch.load(path))


lstm_model = LSTMModel()
# lstm_model.train()

lstm_model.load()
# print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

##add weight decay
##add embedding dropout

saved lstm_predictions.npy


Run the cell below in order to train your improved LSTM and evaluate it.  

In [ ]:
## Feel free to copy your original LSTM solution down here to modify for your report if you'd like. 
# YOUR CODE [optionally] HERE
##

#lstm_model = LSTMModel()
#lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

### Submission

Upload a submission with the following files to Gradescope:
* hw1b.ipynb (rename to match this exactly)
* lstm_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* bigram_predictions.npy
* report.pdf

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.